In [14]:
from __future__ import absolute_import, division, print_function, unicode_literals
import os
import sys
# turn off GPU
# os.environ["CUDA_VISIBLE_DEVICES"]="-1" 

import tensorflow as tf

import numpy as np
import pathlib
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.animation as animation
from IPython.display import HTML

In [15]:
L=8
Tmin=3.00
Tmax=9.00
# Tmin=4.00
# Tmax=8.00
# en=8 #give an error
en=10

number_of_ensembles=500*en

In [16]:
path_QCP = '/pds/pds132/ML/FNN/'
file_QCP_image = path_QCP + 'QJMC_{L:d}/QJMC_AC_1000_{L:d}_{T:0.2f}_500_{en:d}.table'
file_QCP_label = path_QCP + 'label/QJMC_AC_1000_12_{T:0.2f}_500_1_isOrdered.table'

In [17]:
number_of_training_files_for_each_phase = int(Tmin/0.04)
number_of_training_files = 2 * number_of_training_files_for_each_phase
NUM_TOTAL = number_of_ensembles*number_of_training_files
NUM_TRAIN = int(NUM_TOTAL*0.9)
NUM_VALID = NUM_TOTAL-NUM_TRAIN
SHUFFLE_BUFFER_SIZE = 1500
BATCH_SIZE = 100

def list_of_file(type_of_data, T_init, T_stop, e):
    list = []
    for T in np.arange(T_init, T_stop, 0.04):
        list.append(type_of_data.format(L=L, T=T, en=e))
    return list

list_of_image=[]; list_of_label=[];
for e in range(1, en+1):
        list_of_image += list_of_file(file_QCP_image, 0.00, Tmin, e)
        list_of_image += list_of_file(file_QCP_image, Tmax, 12.00, e)
        list_of_label += list_of_file(file_QCP_label, 0.00, Tmin, e)
        list_of_label += list_of_file(file_QCP_label, Tmax, 12.00, e)

images = tf.data.Dataset.from_tensor_slices(list_of_image).interleave(tf.data.TextLineDataset, cycle_length=number_of_training_files, num_parallel_calls=tf.data.experimental.AUTOTUNE)
labels = tf.data.Dataset.from_tensor_slices(list_of_label).interleave(tf.data.TextLineDataset, cycle_length=number_of_training_files, num_parallel_calls=tf.data.experimental.AUTOTUNE)
dataset = tf.data.Dataset.zip((images, labels))

In [18]:
# split training and validation
dataset_train = dataset.take(NUM_TRAIN)
dataset_valid = dataset.skip(NUM_TRAIN).take(NUM_VALID)
# repeat and shuffle
dataset_train = dataset_train.repeat().shuffle(SHUFFLE_BUFFER_SIZE)

In [19]:
# map and cache
@tf.function
def decoder(image, label):
    image = tf.strings.split(image)
    image = tf.strings.to_number(image, out_type = tf.float64)
#     image = tf.reshape(image, shape = [-1])
    label = tf.strings.split(label)
    label = tf.strings.to_number(label, out_type = tf.float64)
#     label = tf.reshape(label, shape = [-1])
    return image, label

dataset_train = dataset_train.map(decoder, num_parallel_calls = tf.data.experimental.AUTOTUNE).cache()
dataset_valid = dataset_valid.map(decoder, num_parallel_calls = tf.data.experimental.AUTOTUNE).cache()

# batch and prefetch
dataset_train = dataset_train.batch(BATCH_SIZE, drop_remainder = True).prefetch(buffer_size = tf.data.experimental.AUTOTUNE)
dataset_valid = dataset_valid.batch(NUM_VALID).prefetch(buffer_size = tf.data.experimental.AUTOTUNE)

In [2]:
images = tf.keras.Input(shape=L, dtype=tf.float64)
x = tf.keras.layers.Reshape((L, 1)) (images)
x = tf.keras.layers.Conv1D(128, 2, activation=tf.keras.activations.relu)(x)
# x = tf.keras.layers.MaxPooling2D()(x)
x = tf.keras.layers.Conv1D(128, 2, activation=tf.keras.activations.relu)(x)
# x = tf.keras.layers.Conv1D(256, 2, activation=tf.keras.activations.relu)(x)
# x = tf.keras.layers.Conv1D(256, 2, activation=tf.keras.activations.relu)(x)
# x = tf.keras.layers.MaxPooling2D()(x)
x = tf.keras.layers.Flatten()(x)
# x = tf.keras.layers.Dense(int(np.power(L * L, 1.0)), activation=tf.keras.activations.relu, kernel_regularizer=tf.keras.regularizers.l2(0.001))(x)
x = tf.keras.layers.Dense(int(np.power(L * L, 1.0)), activation=tf.keras.activations.relu)(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dense(int(np.power(L * L, 0.9)), activation=tf.keras.activations.relu)(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dense(int(np.power(L * L, 0.8)), activation=tf.keras.activations.relu)(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dense(int(np.power(L * L, 0.7)), activation=tf.keras.activations.relu)(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dense(int(np.power(L * L, 0.6)), activation=tf.keras.activations.relu)(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dense(int(np.power(L * L, 0.5)), activation=tf.keras.activations.relu)(x)
x = tf.keras.layers.BatchNormalization()(x)
labels = tf.keras.layers.Dense(2, activation=tf.keras.activations.softmax)(x)

cnn = tf.keras.Model(images, labels)
# cnn.compile(optimizer=tf.keras.optimizers.RMSprop(),
cnn.compile(optimizer=tf.keras.optimizers.Adam(),
#         loss=tf.keras.losses.MeanSquaredError(),
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=['accuracy'])
cnn.summary()
tf.keras.utils.plot_model(cnn, 
                          to_file='dataTensorFlow_ising.png',
                          show_shapes=True,
                          show_layer_names=False,
                          rankdir='LR')

In [3]:
cnn_history = cnn.fit(dataset_train,
                      verbose=1,
                    #   epochs=30,
                    #   epochs=25,
                    #   epochs=35,
                       epochs=5,
                    #   epochs=1,
                      steps_per_epoch=int(NUM_TRAIN/BATCH_SIZE),
                      validation_data=dataset_valid,
                      validation_steps=1)

In [ ]:
def plot_history(history, filename):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch

    plt.clf()
    plt.subplot(2, 1, 1)
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.plot(hist['epoch'], hist['accuracy'], label='Train')
    plt.plot(hist['epoch'], hist['val_accuracy'], label = 'Validation')
    plt.legend()
    
    plt.subplot(2, 1, 2)
    plt.xlabel('Epoch')
    plt.ylabel('loss')
    plt.plot(hist['epoch'], hist['loss'], label='Train')
    plt.plot(hist['epoch'], hist['val_loss'], label = 'Validation')
    plt.legend()

    plt.tight_layout()
    plt.savefig(filename)

save = '/pds/pds181/jmj/JupyterLab/Project_QCP_criticalPoint/RNN/Saves/QCP4_{L:d}_{Tmin:0.2f}_{Tmax:0.2f}_{en:d}_save.h5'
struct = '/pds/pds181/jmj/JupyterLab/Project_QCP_criticalPoint/RNN/Saves/QCP4_{L:d}_{Tmin:0.2f}_{Tmax:0.2f}_{en:d}_struct.png'
train_history = '/pds/pds181/jmj/JupyterLab/Project_QCP_criticalPoint/RNN/Saves/QCP4_{L:d}_{Tmin:0.2f}_{Tmax:0.2f}_{en:d}_history.pdf'

tf.keras.utils.plot_model(cnn, to_file = struct.format(L = L, Tmin = Tmin, Tmax = Tmax, en = en), show_shapes = True, show_layer_names = False, rankdir = 'LR')
cnn.save(save.format(L = L, Tmin = Tmin, Tmax = Tmax, en = en))
plot_history(cnn_history, train_history.format(L = L, Tmin = Tmin, Tmax = Tmax, en = en))

In [ ]:
# file_QCP_TEST_image = path_QCP + 'QJMC_{L:d}/QJMC_AC_1000_{L:d}_{T:0.2f}_500_{en:d}.table'
number_of_test_files = int((Tmax-Tmin)/0.04)
number_of_total_files=number_of_test_files+number_of_training_files   
      
list_of_image=[]; list_of_label=[];
for e in range(1, en+1):
    list_of_image += list_of_file(file_QCP_image, 0.00, 12.00, e)
    list_of_label += list_of_file(file_QCP_label, 0.00, 12.00, e)

images = tf.data.Dataset.from_tensor_slices(list_of_image).interleave(tf.data.TextLineDataset, cycle_length=number_of_total_files, num_parallel_calls=tf.data.experimental.AUTOTUNE)
labels = tf.data.Dataset.from_tensor_slices(list_of_label).interleave(tf.data.TextLineDataset, cycle_length=number_of_total_files, num_parallel_calls=tf.data.experimental.AUTOTUNE)
dataset = tf.data.Dataset.zip((images, labels))


# map and caching
@tf.function
def decoder(image, label):
    image = tf.strings.split(image)
    image = tf.strings.to_number(image, out_type = tf.float64)
#     image = tf.reshape(image, shape = [-1])
    label = tf.strings.split(label)
    label = tf.strings.to_number(label, out_type = tf.float64)
    return image, label

dataset = dataset.map(decoder, num_parallel_calls = tf.data.experimental.AUTOTUNE)
dataset = dataset.batch(number_of_total_files)

In [ ]:
output = cnn.predict(dataset)

In [ ]:
outputs = np.zeros((number_of_total_files, 2))

NUM_TEST=int(number_of_ensembles * number_of_total_files)

for i in range(NUM_TEST):
    outputs[i%(number_of_total_files)] += output[i]
outputs/=(NUM_TEST/(number_of_total_files))
fig = plt.figure()
Tc=6.0
plt.xlabel('Temperature')
plt.ylabel('Output Layer')
plt.axvspan(0.0, Tmin, color = 'C0', alpha = 0.3)
plt.axvspan(Tmin+number_of_test_files*0.04, 12.00, color = 'C3', alpha = 0.3)

plt.axvline(x = Tc, ls='--', color = 'C0', linewidth=2)
plt.axhline(0.5, ls='--', color = 'C0', linewidth=2)

outputs_withT = np.zeros(((number_of_total_files), 3))
for i in range(number_of_total_files):
    T = 0 + 0.04 * i
    outputs_withT[i][0] = T
    outputs_withT[i][1] = outputs[i][0]
    outputs_withT[i][2] = outputs[i][1]
     
plt.plot([outputs_withT[i][0] for i in range(number_of_total_files)], [outputs_withT[i][1] for i in range(number_of_total_files)])
plt.plot([outputs_withT[i][0] for i in range(number_of_total_files)], [outputs_withT[i][2] for i in range(number_of_total_files)])

fig = '/pds/pds181/jmj/JupyterLab/Project_QCP_criticalPoint/RNN/Results/QCP4_{L:d}_{Tmin:0.2f}_{Tmax:0.2f}_{en:d}.pdf'
plt.savefig(fig.format(L = L, Tmin = Tmin, Tmax = Tmin+number_of_test_files*0.04, en = en))

dat = '/pds/pds181/jmj/JupyterLab/Project_QCP_criticalPoint/RNN/Results/QCP4_{L:d}_{Tmin:0.2f}_{Tmax:0.2f}_{en:d}.table'
np.savetxt(dat.format(L = L,  Tmin = Tmin, Tmax = Tmin+number_of_test_files*0.04, en = en), outputs_withT, fmt = "%f") 

In [ ]:
#### import matplotlib as mpl
plt.figure()
Tc=6.0
Tmin=3.0
Tmax=9.0
plt.xlabel('Quantum coherence')
plt.ylabel('Output Layer')
plt.axvspan(0.0, Tmin, color = 'C0', alpha = 0.3)
plt.axvspan(Tmax, 12.00, color = 'C3', alpha = 0.3)

plt.axvline(x = Tc, ls='--', color = 'C0', linewidth=2)
plt.axhline(0.5, ls='--', color = 'C0', linewidth=2)

# f = open('%s.csv' % name, 'wb')/pds/pds181/jmj/JupyterLab/Project_QCP_criticalPoint/FNN/Results/
filename_test='/pds/pds181/jmj/JupyterLab/Project_QCP_criticalPoint/RNN/Results/QCP4_{L:d}_{Tmin:0.2f}_{Tmax:0.2f}_{en:d}.table'
# cmap = mpl.cm.autumn
colors = ["red", "orange", "blue", "green", "purple", "yellow", "black","red", "orange", "blue", "green", "purple", "yellow", "black"]


X, Y, Z = [], [], []
for M in range(8, 17, 2):
    for line in open(filename_test.format(L = M, Tmin = Tmin, Tmax = Tmax, en = en), 'r'):
        values = [float(s) for s in line.split()]
        X.append(values[0])
        Y.append(values[1])  
        Z.append(values[2])  
    plt.plot(X, Y, color='{col:s}'.format(col = colors[M-8]), label='{L:d}'.format(L = M))
    plt.plot(X, Z, color='{col:s}'.format(col = colors[M-8]))
    X, Y, Z = [], [], []
plt.legend()

X, Y, Z = [], [], []
for line in open('/pds/pds181/jmj/JupyterLab/Project_QCP_criticalPoint/RNN/Results/QCP4_18_4.00_8.00_10.table', 'r'):
    values = [float(s) for s in line.split()]
    X.append(values[0])
    Y.append(values[1])  
    Z.append(values[2])  
plt.plot(X,Y, color='g', label='18')
plt.plot(X,Z, color='g')
plt.legend()